In [ ]:
# Import stuff
from os.path import join
from pandas import DataFrame

from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.base import Bunch
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.modelgen import SpecifyModel
from nipype.interfaces.fsl.model import Level1Design, FEATModel, FILMGLS, GLM
from nipype.interfaces.fsl.preprocess import FLIRT, SUSAN

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/home/camachocm2/Analysis/ChEC/fmri_proc'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevel'
workflow_dir = analysis_home + '/workflows'

#subject_info = DataFrame.read_csv(analysis_home + '/../misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
subjects_list = ['pilot002']

# data collection specs
TR = 0.8 #in seconds
num_slices = 40
slice_direction = False #True = z direction, top to bottom
interleaved = True

conditionNames = ['tense', 'negative', 'positive']
contrasts_list = [['tense', 'T', conditionNames, [1, 0, 0, 0]], 
                  ['negative', 'T', conditionNames, [0, 1, 0, 0]], 
                  ['positive', 'T', conditionNames, [0, 0, 1, 0]],
                  ['negative>positive', 'T', conditionNames, []]
                 ]  

In [ ]:
# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Pull files
file_template = {'preproc_func': preproc_dir + 'coreg_func/{subjid}/preproc_func.nii.gz'}

selectfunc = Node(SelectFiles(file_template), name='selectfunc')

# Sink data of interest (mostly for QC)
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = preproc_dir,
                        container = preproc_dir,
                        substitutions = substitutions), 
                name='datasink')

In [ ]:
## Timing handling nodes
def affectiveTiming(timing_file, motion):
    from nipype.interfaces.base import Bunch
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    conditionNames = ['ZeroBack', 'OneBack']
    onsets = [[15, 118, 222, 326], [66, 170, 274]]
    durations = [[36, 37, 37, 37], [37, 37, 37]]
    
    tasktiming = []
    tasktiming.insert(0,Bunch(conditions=conditionNames,
                              onsets=onsets,
                              durations=durations,
                              amplitudes=None,
                              tmod=None,
                              pmod=None,
                              regressor_names=None,
                              regressors=None))

    return(tasktiming)

In [ ]:
getSubjTaskInfo = Node(name='getSubjTaskInfo',
                       interface=Function(input_names=[],
                                          output_names=['tasktiming'],
                                          function=affectiveTiming))

# Specify FSL model
modelspec = Node(SpecifyModel(time_repetition=TR, 
                              high_pass_filter_cutoff=128, 
                              input_units='secs'),
                 name='modelspec')

# Generate a level 1 design
level1design = Node(Level1Design(bases={'dgamma':{'derivs': False}},
                                 interscan_interval=TR, # the TR
                                 model_serial_correlations=True,
                                 contrasts=contrasts_list), 
                    name='level1design')

# Estimate Level 1
generateModel = Node(FEATModel(), 
                     name='generateModel')

#run the GLM
estmodel = Node(GLM(out_file = 'betas.nii.gz', 
                    out_cope='cope.nii.gz',
                    out_t_name = 'tstat.nii.gz'), 
                name= 'estmodel')

In [ ]:
L1workflow = Workflow(name='L1workflow')
L1workflow.connect([(infosource,selectfiles,[('subjid','subjid')]),
                    (selectfiles,modelspec,[('motion','realignment_parameters')]), 
                    (selectfiles,modelspec,[('outliers','outlier_files')]),
                    (selectfiles,modelspec,[('func','functional_runs')]),
                    (getSubjTaskInfo,modelspec,[('tasktiming','subject_info')]),
                    (modelspec,level1design,[('session_info','session_info')]),
                    (level1design,generateModel,[('ev_files','ev_files')]),
                    (level1design,generateModel,[('fsf_files','fsf_file')]),
                    (generateModel,estmodel,[('design_file','design')]),
                    (generateModel,estmodel,[('con_file','contrasts')]),
                    (selectfiles,estmodel,[('func','in_file')]),
                    
                    (generateModel, datasink, [('design_image','design_image')]),
                    (estmodel, datasink, [('out_cope','copes')]),
                    (estmodel, datasink, [('out_t','tstats')]),
                    (estmodel, datasink, [('out_file','betas')])
                   ])
L1workflow.base_dir = join(workflow_dir)
L1workflow.write_graph(graph2use='flat')
L1workflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})